* 1st: compute position selectivity

In [1]:
import glob
import numpy as np
from typing import Dict, List
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.trials.spikes import firing_rate
from ephysvibe.structures.population_data import PopulationData
from ephysvibe.trials import select_trials
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
from ephysvibe.stats import smetrics
import os

In [2]:
def get_selectivity_info(neu: NeuronData):
    win = 100
    res = {}
    res["nid"] = neu.get_neuron_id()
    samples = [11, 15, 51, 55, 0]

    inout_fr = []
    for inout in ["in", "out"]:
        mask = getattr(neu, "mask_" + inout)
        sp = getattr(neu, "sample_on_" + inout)

        fr = firing_rate.moving_average(data=sp, win=100, step=1)[:, :-win]  #
        sample_id = neu.sample_id[mask]
        fr_samples = select_trials.get_sp_by_sample(fr, sample_id, samples)

        sample = np.concatenate(
            (fr_samples["11"], fr_samples["15"], fr_samples["51"], fr_samples["55"],fr_samples["0"])
        )
        inout_fr.append(sample)

    pos_lat, pos_score, pos_p = smetrics.get_selectivity(
        inout_fr[0], inout_fr[1], win=75, scores=True
    )
    pos_selec = (
        np.nan
        if np.isnan(pos_lat)
        else "in" if pos_score[pos_lat] > 0 else "out"
    )


    res["pos_lat"] = pos_lat
    res["pos_selec"] = pos_selec
    res["pos_score"] = pos_score
    res["pos_p"] = pos_p

    return res


In [3]:
def get_neu_align(path, params, sp_sample=False):

    neu = NeuronData.from_python_hdf5(path)
    for it in params:
        sp, mask = neu.align_on(
            select_block=it["select_block"],
            select_pos=it["inout"],
            event=it["event"],
            time_before=it["time_before"],
            error_type=0,
        )
        endt = it["time_before"] + it["end"]
        stt = it["time_before"] + it["st"]
        setattr(neu, it["sp"], np.array(sp[:, :endt], dtype=it["dtype_sp"]))
        setattr(neu, it["mask"], np.array(mask, dtype=it["dtype_mask"]))
        setattr(neu, "st_" + it["event"] + "_" + it["inout"], np.array(stt, dtype=int))
        setattr(
            neu,
            "time_before_" + it["event"] + "_" + it["inout"],
            np.array(it["time_before"], dtype=int),
        )

    if ~sp_sample:
        setattr(neu, "sp_samples", np.array([]))

    return neu

In [5]:
# Define parameters
areas = ["lip"]
subject = "Riesling"
# paths
filepaths = {
    "lip": "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure/session_struct/lip/neurons/",
    "pfc": "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure/session_struct/pfc/neurons/",
    "v4": "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure/session_struct/v4/neurons/",
}
savepath = "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/activation_index/"
popu_path = {
    # "lip": "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/selectivity/population_selectivity_lip.h5",
    # "pfc": "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/selectivity/population_selectivity_pfc.h5",
    # "v4": "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/selectivity/population_selectivity_v4.h5",
}
for area in areas:
    print(area)
    if not bool(popu_path):
        path = filepaths[area]
        neu_path = path + "*neu.h5"
        path_list = glob.glob(neu_path)

        params = [
            {
                "inout": "in",
                "sp": "sample_on_in",
                "mask": "mask_in",
                "event": "sample_on",
                "time_before": 300,
                "st": 0,
                "end": 1550,
                "select_block": 1,
                "win": 100,
                "dtype_sp": np.int8,
                "dtype_mask": bool,
            },
            {
                "inout": "out",
                "sp": "sample_on_out",
                "mask": "mask_out",
                "event": "sample_on",
                "time_before": 300,
                "st": 0,
                "end": 1550,
                "select_block": 1,
                "win": 100,
                "dtype_sp": np.int8,
                "dtype_mask": bool,
            },
        ]
        population_list = Parallel(n_jobs=-1)(
            delayed(get_neu_align)(neu, params) for neu in tqdm(path_list)
        )
        comment = str(params)
        population = PopulationData(population_list, comment=comment)
        print("Saving population.h5")
        population.to_python_hdf5(savepath + "population_selectivity_" + area + ".h5")
        population = PopulationData.from_python_hdf5(
            savepath + "population_selectivity_" + area + ".h5"
        )
    else:
        print("Reading population data")
        population = PopulationData.from_python_hdf5(popu_path[area])
    print("Computing selectivity")
    df_selectivity = population.execute_function(
        get_selectivity_info, n_jobs=-1, ret_df=True
    )

    df_selectivity.to_pickle(savepath + "population_selectivity_" + area + ".pkl")

lip


  0%|          | 0/530 [00:00<?, ?it/s]

100%|██████████| 530/530 [11:46<00:00,  1.33s/it]


Saving population.h5


FileNotFoundError: [Errno 2] Unable to synchronously create file (unable to open file: name = '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/activation_index/population_selectivity_lip.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [7]:
population.to_python_hdf5(savepath + "population_selectivity_" + area + ".h5")
population = PopulationData.from_python_hdf5(
    savepath + "population_selectivity_" + area + ".h5"
)